##### Importing libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%matplotlib inline
#pd.options.display.float_format = '{:,.2f}'.format
#import warnings
#warnings.filterwarnings('ignore')

##### Importing data

In [4]:
df = pd.read_csv('loan.csv')

/Users/chandrahas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [6]:
# Shape of Dataframe

df.shape

(39717, 111)

##### Data Cleaning

Checking column which contribute to the analysis by looking to the Data dictionary; those who do not seems to contribute to the analysis can be removed from the dataset.

In [7]:
# 'id' and 'member_id' are just random number and do not contribute to analysis; 'url' also doesn't contribute to analysis
# dropping columns

df = df.drop(['id','member_id', 'url'], axis=1)

In [8]:
#Checking column with missing values

null = df.isnull().sum()
null_cent = (null/len(df))*100
null_chk = pd.concat([null, null_cent], axis=1)
null_chk[null_chk[1]>0]

,0,1
emp_title,2459,6.191303
emp_length,1075,2.706650
desc,12940,32.580507
title,11,0.027696
mths_since_last_delinq,25682,64.662487
...,...,...
tax_liens,39,0.098195
tot_hi_cred_lim,39717,100.000000
total_bal_ex_mort,39717,100.000000
total_bc_limit,39717,100.000000


In [9]:
#columns with more than 50% missing values  

missing_col = null_chk[null_chk[1]>50]
missing_col.head()

,0,1
mths_since_last_delinq,25682,64.662487
mths_since_last_record,36931,92.985372
next_pymnt_d,38577,97.129693
mths_since_last_major_derog,39717,100.000000
annual_inc_joint,39717,100.000000


In [10]:
len(missing_col)

57

Lets remove the columns with more than 50% missing values

In [11]:
#Dropping columns

df = df.drop(list(missing_col.index),axis=1)

In [12]:
# checking shape of the Dataframe post removing columns with missing values

df.shape

(39717, 51)

In [13]:
chk = (df.isnull().sum()/len(df))*100
chk[chk>0]

emp_title                      6.191303
emp_length                     2.706650
desc                          32.580507
title                          0.027696
revol_util                     0.125891
last_pymnt_d                   0.178765
last_credit_pull_d             0.005036
collections_12_mths_ex_med     0.140998
chargeoff_within_12_mths       0.140998
pub_rec_bankruptcies           1.754916
tax_liens                      0.098195
dtype: float64

##### Identify which feature needs to be fixed and which can be removed

***'desc'*** = Loan description provided by the borrower

In [14]:
#desc is the Loan description provided by the borrower; this can be also concluded from the title column. 
#Therefore we can remove this column

#Dropping desc

df = df.drop('desc', axis=1)

***'emp_title'*** = The job title supplied by the Borrower when applying for the loan

In [15]:
print(df['emp_title'].value_counts())
print("\n")
print('Unique values in emp_title: ',df['emp_title'].nunique())

US Army                                     134
Bank of America                             109
IBM                                          66
AT&T                                         59
Kaiser Permanente                            56
                                           ... 
Rushmore Capital Inc                          1
Chesapeake Veterinary Surgical Specialis      1
Northland Area Federal CU                     1
Wayne County Airport Authority                1
Medix Staffing Solutions, Inc.                1
Name: emp_title, Length: 28820, dtype: int64


Unique values in emp_title:  28820


In [16]:
# emp_title has large unique value and its a categorical data; it wont contribute to the Analysis, so drop the column

# Dropping emp_title

df = df.drop('emp_title', axis=1)

***'emp_length'*** = Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and
10 means ten or more years.

In [17]:
print(df['emp_length'].value_counts())
print("\n")
print('Unique values in emp_length: ',df['emp_length'].nunique())
print('Total missing values: ',df['emp_length'].isnull().sum())

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
Name: emp_length, dtype: int64


Unique values in emp_length:  11
Total missing values:  1075


In [18]:
# emp_length can be used for analysis and fixing null values will add bias to the data

# removing rows from emp_length column which contains null values

df = df[~(df['emp_length'].isnull())]

***'title'*** = The loan title provided by the borrower

In [19]:
print(df['title'].value_counts())
print("\n")
print('Unique values in title: ',df['title'].nunique())
print('Total missing values: ',df['title'].isnull().sum())

Debt Consolidation                   2144
Debt Consolidation Loan              1691
Personal Loan                         641
Consolidation                         511
debt consolidation                    486
                                     ... 
Bike payed off                          1
Debt/Unexpected                         1
Payoff High Interest Credit Cards       1
skiteam7                                1
Tickets Advertising                     1
Name: title, Length: 19148, dtype: int64


Unique values in title:  19148
Total missing values:  10


In [20]:
# title can be used for analysis, therefore missing values needs to be removed

# removing rows with missing values

df = df[~(df['title'].isnull())]

***'revol_util'*** = Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

In [21]:
print(df['revol_util'].value_counts())
print("\n")
print('Unique values in revol_util: ',df['revol_util'].nunique())
print('Total missing values: ',df['revol_util'].isnull().sum())

0%        941
0.20%      62
63%        61
40.70%     57
66.70%     57
         ... 
26.33%      1
1.88%       1
32.04%      1
13.56%      1
0.12%       1
Name: revol_util, Length: 1087, dtype: int64


Unique values in revol_util:  1087
Total missing values:  47


In [22]:
# removing rows from revol_util which contains missing values

df = df[~(df['revol_util'].isnull())]

***'last_pymnt_d'*** = Last month payment was received

In [23]:
print(df['last_pymnt_d'].value_counts())
print("\n")
print('Unique values in last_pymnt_d: ',df['last_pymnt_d'].nunique())
print('Total missing values: ',df['last_pymnt_d'].isnull().sum())

May-16    1209
Mar-13     999
Dec-14     910
May-13     883
Feb-13     845
          ... 
Nov-08      10
Jun-08      10
Mar-08       5
Jan-08       4
Feb-08       1
Name: last_pymnt_d, Length: 101, dtype: int64


Unique values in last_pymnt_d:  101
Total missing values:  62


In [24]:
# Imputing data to the column will add bias

# removing rows from last_pymnt_d which contains missing values

df = df[~(df['last_pymnt_d'].isnull())]

***'last_credit_pull_d'*** = The most recent month LC pulled credit for this loan

In [25]:
print(df['last_credit_pull_d'].value_counts())
print("\n")
print('Unique values in last_credit_pull_d: ',df['last_credit_pull_d'].nunique())
print('Total missing values: ',df['last_credit_pull_d'].isnull().sum())

May-16    10031
Apr-16     2476
Mar-16     1094
Feb-13      831
Feb-16      714
          ...  
Jun-08        1
May-07        1
May-08        1
Jul-07        1
Jul-08        1
Name: last_credit_pull_d, Length: 106, dtype: int64


Unique values in last_credit_pull_d:  106
Total missing values:  2


In [26]:
# There are only two missing values which can be deleted

# Deleting rows with missing values

df = df[~(df['last_credit_pull_d'].isnull())]

***'collections_12_mths_ex_med'*** = Number of collections in 12 months excluding medical collections

In [27]:
print(df['collections_12_mths_ex_med'].value_counts())
print("\n")
print('Unique values in collections_12_mths_ex_med: ',df['collections_12_mths_ex_med'].nunique())
print('Total missing values: ',df['collections_12_mths_ex_med'].isnull().sum())

0.0    38465
Name: collections_12_mths_ex_med, dtype: int64


Unique values in collections_12_mths_ex_med:  1
Total missing values:  56


In [28]:
# Dropping column

df = df.drop('collections_12_mths_ex_med', axis=1)

***'chargeoff_within_12_mths'*** = Number of charge-offs within 12 months

In [29]:
print(df['chargeoff_within_12_mths'].value_counts())
print("\n")
print('Unique values in chargeoff_within_12_mths: ',df['chargeoff_within_12_mths'].nunique())
print('Total missing values: ',df['chargeoff_within_12_mths'].isnull().sum())

0.0    38465
Name: chargeoff_within_12_mths, dtype: int64


Unique values in chargeoff_within_12_mths:  1
Total missing values:  56


In [30]:
# The column only has only one value, therefore the column can be removed
# Dropping the column 

df = df.drop('chargeoff_within_12_mths', axis=1)

***'pub_rec_bankruptcies'*** = Number of public record bankruptcies

In [31]:
print(df['pub_rec_bankruptcies'].value_counts())
print("\n")
print('Unique values in pub_rec_bankruptcies: ',df['pub_rec_bankruptcies'].nunique())
print('Total missing values: ',df['pub_rec_bankruptcies'].isnull().sum())

0.0    36262
1.0     1556
2.0        7
Name: pub_rec_bankruptcies, dtype: int64


Unique values in pub_rec_bankruptcies:  3
Total missing values:  696


pub_rec_bankruptcies has 1.75% of missing rows which is a small percentage, therefore the Number of public record bankruptcies can be imputed with the mode of the column

In [32]:
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].mode()[0])

***'tax_liens'*** = Number of tax liens 

In [33]:
print(df['tax_liens'].value_counts())
print("\n")
print('Unique values in tax_liens: ',df['tax_liens'].nunique())
print('Total missing values: ',df['tax_liens'].isnull().sum())

0.0    38482
Name: tax_liens, dtype: int64


Unique values in tax_liens:  1
Total missing values:  39


In [34]:
# The column only has only one value, therefore the column can be removed
# dropping the column

df = df.drop('tax_liens', axis=1)

Dataframe post removal of missing values

In [35]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,policy_code,application_type,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies
0,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,...,0.00,0.00,Jan-15,171.62,May-16,1,INDIVIDUAL,0,0,0.0
1,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,...,117.08,1.11,Apr-13,119.66,Sep-13,1,INDIVIDUAL,0,0,0.0
2,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,...,0.00,0.00,Jun-14,649.91,May-16,1,INDIVIDUAL,0,0,0.0
3,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,...,0.00,0.00,Jan-15,357.48,Apr-16,1,INDIVIDUAL,0,0,0.0
4,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,...,0.00,0.00,May-16,67.79,May-16,1,INDIVIDUAL,0,0,0.0


In [36]:
df.shape

(38521, 46)

Checking features which has only one value in the column

In [37]:
df.nunique().sort_values().head(10)

acc_now_delinq          1
application_type        1
policy_code             1
initial_list_status     1
delinq_amnt             1
pymnt_plan              1
term                    2
pub_rec_bankruptcies    3
verification_status     3
loan_status             3
dtype: int64

In [38]:
# Dropping columns having only one value

df = df.drop(['acc_now_delinq', 'application_type', 'policy_code', 'initial_list_status',
              'delinq_amnt', 'pymnt_plan'], axis=1)

Checking and evaluating each features with their values and data types

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38521 entries, 0 to 39716
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                38521 non-null  int64  
 1   funded_amnt              38521 non-null  int64  
 2   funded_amnt_inv          38521 non-null  float64
 3   term                     38521 non-null  object 
 4   int_rate                 38521 non-null  object 
 5   installment              38521 non-null  float64
 6   grade                    38521 non-null  object 
 7   sub_grade                38521 non-null  object 
 8   emp_length               38521 non-null  object 
 9   home_ownership           38521 non-null  object 
 10  annual_inc               38521 non-null  float64
 11  verification_status      38521 non-null  object 
 12  issue_d                  38521 non-null  object 
 13  loan_status              38521 non-null  object 
 14  purpose               

In [40]:
df.select_dtypes(include=['object']).head()

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,last_pymnt_d,last_credit_pull_d
0,36 months,10.65%,B,B2,10+ years,RENT,Verified,Dec-11,Fully Paid,credit_card,Computer,860xx,AZ,Jan-85,83.70%,Jan-15,May-16
1,60 months,15.27%,C,C4,< 1 year,RENT,Source Verified,Dec-11,Charged Off,car,bike,309xx,GA,Apr-99,9.40%,Apr-13,Sep-13
2,36 months,15.96%,C,C5,10+ years,RENT,Not Verified,Dec-11,Fully Paid,small_business,real estate business,606xx,IL,Nov-01,98.50%,Jun-14,May-16
3,36 months,13.49%,C,C1,10+ years,RENT,Source Verified,Dec-11,Fully Paid,other,personel,917xx,CA,Feb-96,21%,Jan-15,Apr-16
4,60 months,12.69%,B,B5,1 year,RENT,Source Verified,Dec-11,Current,other,Personal,972xx,OR,Jan-96,53.90%,May-16,May-16


In [41]:
# fetching int_rate by removing % and converting it to float

df['int_rate'] = df['int_rate'].apply(lambda x: float(x[:-1]))

In [42]:
df['zip_code'].head()

0    860xx
1    309xx
2    606xx
3    917xx
4    972xx
Name: zip_code, dtype: object

In [43]:
# this column doesn't have full zip code and therefore can be dropped

df = df.drop('zip_code', axis=1)

In [44]:
# fetching revol_util by removing % and converting it to float

df['revol_util'] = df['revol_util'].apply(lambda x: float(x[:-1]))

In [45]:
df.shape

(38521, 39)

In [49]:
df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv']]

,loan_amnt,funded_amnt,funded_amnt_inv
0,5000,5000,4975.0
1,2500,2500,2500.0
2,2400,2400,2400.0
3,10000,10000,10000.0
4,3000,3000,3000.0
...,...,...,...
39712,2500,2500,1075.0
39713,8500,8500,875.0
39714,5000,5000,1325.0
39715,5000,5000,650.0


In [52]:
df[df['loan_amnt']!=df['funded_amnt']].shape

(1815, 39)

In [53]:
df[df['loan_amnt']!=df['funded_amnt_inv']].shape

(19769, 39)